## Домашнее задание к лекции "Основы веб-скрапинга и работы с API" ##

### Задание 1. ###
**Обязательная часть**

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

$KEYWORDS = ['python', 'парсинг']$

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

**Дополнительная часть (необязательная)**

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>


In [1]:
import requests

In [2]:
import pandas as pd

In [3]:
from bs4 import BeautifulSoup

In [4]:
import time
time.sleep(0.2)

In [5]:
keywords = ['дизайн игр', 'конференции', 'математика']

In [6]:
URL = 'https://habr.com/ru/all/'

In [7]:
res = requests.get(URL)
soup = BeautifulSoup(res.text, 'html.parser')

In [8]:
# добираемся до блоков с новостями

posts = soup.find_all('article', class_='post post_preview')

In [67]:
# добавляем извлечение хабов из постов, чтобы отбирать только нужные
kom_news = pd.DataFrame()

for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    hubs = post.find_all('a', class_='hub-link')
    for hub in hubs:
            hub_lower = hub.text.lower()
           # ищем вхождение хотя бы одного желаемого хаба
            if any([hub_lower in key for key in keywords]):
               # пост нам интересен - делаем с ним все что захотим:
               # можно отправит в телеграм уведомление, можно на почту и т.п.
                date = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date()
                title_element = post.find('a', class_='post__title_link')
                row = {'date': date, 'title': title_element.text, 'link': title_element.attrs.get('href')}
                kom_news = pd.concat([kom_news, pd.DataFrame([row])]) 

In [69]:
kom_news

,date,title,link
0,2020-08-02,Digital-мероприятия в Москве c 3 по 9 августа,https://habr.com/ru/post/513536/
0,2020-08-02,Геймификация быта и оборотная сторона положите...,https://habr.com/ru/post/513394/
0,2020-08-02,Я не понял проблему Гольдбаха,https://habr.com/ru/post/513538/
0,2020-08-02,Гидродинамическое моделирование (CFD) на релье...,https://habr.com/ru/post/513500/


### Дополнительная часть (необязательная)

In [12]:
def get_all_links(url):
    all_refs = []

    res = requests.get(URL)
    time.sleep(0.3)
    soup = BeautifulSoup(res.text, 'html.parser')
    news_blocks = soup.find_all('article', class_='post')
    articles_intro = list(map(lambda x: x.find('h2', class_='post__title'), news_blocks))
    a_list = list(map(lambda x: x.find('a').get('href'), articles_intro))


    return a_list

all_links = get_all_links(URL)
all_links

['https://habr.com/ru/post/513412/',
 'https://habr.com/ru/post/513564/',
 'https://habr.com/ru/company/vdsina/blog/513436/',
 'https://habr.com/ru/post/513558/',
 'https://habr.com/ru/post/513550/',
 'https://habr.com/ru/post/513548/',
 'https://habr.com/ru/post/513536/',
 'https://habr.com/ru/post/513394/',
 'https://habr.com/ru/company/kauri_iot/blog/513540/',
 'https://habr.com/ru/company/flant/blog/512762/',
 'https://habr.com/ru/post/513538/',
 'https://habr.com/ru/company/ozontech/blog/513056/',
 'https://habr.com/ru/post/513500/',
 'https://habr.com/ru/post/513532/',
 'https://habr.com/ru/post/513230/',
 'https://habr.com/ru/post/513526/',
 'https://habr.com/ru/post/513524/',
 'https://habr.com/ru/company/southbridge/blog/513504/',
 'https://habr.com/ru/company/southbridge/blog/513240/',
 'https://habr.com/ru/post/513170/']

In [57]:
news = pd.DataFrame()
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    time.sleep(0.3)
    date = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date()
    
    title = soup.find('span', class_='post__title-text')
    try:
        title = title.text
    except:
        title = title
    text_block = soup.find('div', class_='post__text').text
    for key in keywords:
        if key in text_block:
            row = {'date': date, 'title': title, 'text': text_block}
            news = pd.concat([news, pd.DataFrame([row])])

In [70]:
news

,date,title,text
0,2020-08-03,Digital-мероприятия в Москве c 3 по 9 августа,Подборка мероприятий на неделю\n\nML Party: он...
0,2020-08-02,Как мы в Dropbox перешли с Nginx на Envoy,В этой статье мы будем говорить о нашей старой...


### Задание 2. ###
**Обязательная часть**
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:

$EMAIL = [xxx@x.ru, yyy@y.com]$

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.



In [167]:
URL_2 = 'https://digibody.avast.com/v1/web/leaks'

In [168]:
response = requests.post(URL_2, json={'email': 'xxx@x.ru'})

In [169]:
data_base = response.json()['value']
data_base

[{'leak_id': '037a277e-d484-4c34-8473-904024e489e1',
  'username': 'xxx@x.ru',
  'domain': '',
  'passwords': [{'password': None, 'encrypted': False}],
  'leak_info': {'id': '037a277e-d484-4c34-8473-904024e489e1',
   'date': 1513900800000,
   'title': 'Combolist of 1.4 Billion Credentials',
   'description': 'The proliferation of stolen or leaked databases has given rise to credential stuffing, a fairly simple technique in which criminals load lists of stolen credentials, called combo lists, into automated brute-forcing tools to test credentials en masse. These tools test stolen passwords against thousands of targeted websites and applications until there is a match.\n\nThis particular combo list was likely compiled over time from a variety of public and private breaches. It contains approximately 1.4 billion email and password records. Criminals are actively leveraging this list, along with credential stuffing tools, to gain unauthorized access to targeted websites. This list has been

In [170]:
# просто создать датафрейм сразу не получится, вся нужная инфа содержиться под ключом leak_info

new_list = []
new_dict = {}
for element in data_base:
    new_dict['email'] = element['username']
    new_dict.update(element['leak_info'])
    new_list.append(new_dict)
    new_dict = {}

In [171]:
new_list

[{'email': 'xxx@x.ru',
  'id': '037a277e-d484-4c34-8473-904024e489e1',
  'date': 1513900800000,
  'title': 'Combolist of 1.4 Billion Credentials',
  'description': 'The proliferation of stolen or leaked databases has given rise to credential stuffing, a fairly simple technique in which criminals load lists of stolen credentials, called combo lists, into automated brute-forcing tools to test credentials en masse. These tools test stolen passwords against thousands of targeted websites and applications until there is a match.\n\nThis particular combo list was likely compiled over time from a variety of public and private breaches. It contains approximately 1.4 billion email and password records. Criminals are actively leveraging this list, along with credential stuffing tools, to gain unauthorized access to targeted websites. This list has been made public on a number of hacking forums and paste sites.',
  'number_of_entries': 1335491701,
  'source_references': [],
  'media_references': 

In [172]:
need_table = pd.DataFrame(new_list)[['email', 'id', 'date', 'description']]
need_table.head()

,email,id,date,description
0,xxx@x.ru,037a277e-d484-4c34-8473-904024e489e1,1513900800000,The proliferation of stolen or leaked database...
1,xxx@x.ru,cf7afbbb-7695-4f97-b755-3d63df87828f,1549411200000,"On January 7, 2019, an online user named Sanix..."
2,aleksey1688,ee4ffcf9-e69f-49b7-9c61-70f94fa82ad3,1485820800000,"In March 2016, CDProjektRed.com.com's forum da..."
3,xixixixixixixixixixi,053fe29c-422d-4b69-bce2-1ce4904af697,1487030400000,"In July and August 2016, two criminals execute..."
4,xxx@x.ru,7f6e3b64-9951-458f-b565-8ff02baf1a59,1488931200000,This source has been marked as sensitive due t...


In [160]:
# нужно перевести дату

import datetime

In [173]:
def time_change (column):
    column = str(column)[:-3]
    new_time = datetime.datetime.fromtimestamp(int(column))
    return new_time.strftime('%Y-%m-%d %H:%M:%S')   

In [174]:
need_table['date'] = need_table.date.apply(time_change)
need_table

,email,id,date,description
0,xxx@x.ru,037a277e-d484-4c34-8473-904024e489e1,2017-12-22 03:00:00,The proliferation of stolen or leaked database...
1,xxx@x.ru,cf7afbbb-7695-4f97-b755-3d63df87828f,2019-02-06 03:00:00,"On January 7, 2019, an online user named Sanix..."
2,aleksey1688,ee4ffcf9-e69f-49b7-9c61-70f94fa82ad3,2017-01-31 03:00:00,"In March 2016, CDProjektRed.com.com's forum da..."
3,xixixixixixixixixixi,053fe29c-422d-4b69-bce2-1ce4904af697,2017-02-14 03:00:00,"In July and August 2016, two criminals execute..."
4,xxx@x.ru,7f6e3b64-9951-458f-b565-8ff02baf1a59,2017-03-08 03:00:00,This source has been marked as sensitive due t...
5,xxx@x.ru,71db3f29-006d-4158-a3f0-8102652fbf57,2019-02-06 03:00:00,"On January 7, 2019, an online user named Sanix..."
6,xxx@x.ru,83571809-585b-4fff-a72f-cdf0bc345916,2019-02-06 03:00:00,"On January 7, 2019, an online user named Sanix..."
7,stalkera116,2702007e-b243-4836-b1d6-a2d85f03b469,2017-02-14 03:00:00,"In July and August of 2016, two criminals carr..."
8,xxx@x.ru,7f667060-f7c1-4f48-b93d-ecb4d066ba4b,2016-10-21 03:00:00,"In October of 2013, criminals penetrated Adobe..."
9,xxx@x.ru,b15edc13-85b2-4c53-a54a-fee2d6b861ea,2019-01-29 03:00:00,"On January 7, 2019, an online user named Sanix..."
